In [ ]:
import geopandas
import laspy
from geopandas import GeoDataFrame
import pandas as pd
from shapely.geometry import Point
import numpy as np

In [ ]:
# shapefile post correction
filepath = "/var/data/cgaydon/data/202110_building_val/metadata/post_correction_shapefile/Compilation_detections.shp"
gdf = geopandas.read_file(filepath)
candidate_buildings_classes = ["bati_valide", "bati_auto", "surdetection_pont","surdetection_veget", 
"surdetection_vehicule", "surdetection_autre"] 
gdf = gdf[gdf.classe.isin(candidate_buildings_classes)]
gdf = gdf.reset_index().rename(columns={"index":"shape_index"})
print(len(gdf))
gdf.head()

In [ ]:
# A predicted las - we keep only points previously detected as buuildings by terra-solid -> we must say if we agree with
# what generated the shapefile in the first place. This ignore differences in vegetation efficiently, as well building parts
# that were missed.
# Initial classes: [  1,   2,   6 (detected building, no validation),  19 (valid building),  20 (surdetection, unspecified),
#     21 (building, forgotten), 104, 110 (surdetection, others), 112 (surdetection, vehicule), 114 (surdetection, others), 115 (surdetection, bridges)]
las_filepath = "/var/data/cgaydon/data/202110_building_val/logs/runs/2021-10-19/12-07-08/validation_preds/test_876000_6602000.las"
# las_filepath = '/var/data/cgaydon/data/202110_building_val/trainvaltest/879000_6647000.las'
las = laspy.read(las_filepath)
classification_flag_candidate = [6, 19, 20,110,112,114,115]  # TODO: check what 104 is.
las.points = las[np.isin(las["classification"],classification_flag_candidate)]
lidar_df = pd.DataFrame(las["BuildingsProba"], columns=["BuildingsProba"])
crs = gdf.crs
geometry = [Point(xy) for xy in zip(las.x,las.y)]
lidar_geodf = GeoDataFrame(lidar_df, crs=crs, geometry=geometry)


In [ ]:
# Keep only points that are within a detected shape
lidar_geodf_inside = lidar_geodf.sjoin(gdf, how="inner", predicate='within')
# Aggregate confusion of points from the same shape in a list
lidar_geodf_inside_lists = lidar_geodf_inside.groupby("shape_index")["BuildingsProba"].agg(lambda x: x.tolist())

In [ ]:
# at the end we have several, independant shape (assume no border effect)
lidar_geodf_inside_lists_all_tiles = pd.concat([lidar_geodf_inside_lists, lidar_geodf_inside_lists])

# Now : calculate % of aggrement for these shape (remember: all points were detected as building by MTS)
def contrast(shape_signals):
    arr = np.array(shape_signals)
    arr = np.sum(arr>=.5) / len(arr)
    return arr
lidar_geodf_inside_lists_all_tiles = lidar_geodf_inside_lists_all_tiles.apply(lambda x: contrast(x))
# join back with geometries
df_out = gdf.join(lidar_geodf_inside_lists_all_tiles, on="shape_index",how="left")
# save

In [ ]:
df_out_expl = df_out.dropna()
df_out_expl.explore(column="BuildingsProba", scheme="naturalbreaks", legend=True, k=5)

In [ ]:
# # https://stackoverflow.com/a/66852676/8086033
# # !pip install multiprocesspandas
# from multiprocesspandas import applyparallel
# lidar_df = pd.DataFrame(data = np.array([smaller_las.x, smaller_las.y]).transpose(), columns=["x","y"])
# # lidar_df.applyparallel(lambda x: Point(x),axis=1, num_processes=2)
# %timeit geometry = lidar_df.apply(lambda x: Point(x),axis=1)